In [1]:
from google.colab import drive
drive._mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#!pip install -q keras

In [2]:
!pip install -q pydrive

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [4]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [5]:
!nvidia-smi

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

Wed Dec 15 01:34:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    25W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 372948033938053860
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10843127808
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1666718350497697385
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
 xla_global_id: 416903419]

In [6]:
project_path='/content/drive/My Drive/Machine Learning/UBC/CPSC_503/SemEval22_T7/'

In [ ]:
#cd '/content/drive/My Drive/' 

In [ ]:
#cd $project_path

In [ ]:
ls

In [7]:
import numpy as np
import pandas as pd
import pickle

In [8]:
def tensorify(lst):
    """
    List must be nested list of tensors (with no varying lengths within a dimension).
    Nested list of nested lengths [D1, D2, ... DN] -> tensor([D1, D2, ..., DN)

    :return: nested list D
    """
    # base case, if the current list is not nested anymore, make it into tensor
    if type(lst[0]) != list:
        if type(lst) == torch.Tensor:
            return lst
        elif type(lst[0]) == torch.Tensor:
            return torch.stack(lst, dim=0)
        else:  # if the elements of lst are floats or something like that
            return torch.tensor(lst)
    current_dimension_i = len(lst)
    for d_i in range(current_dimension_i):
        tensor = tensorify(lst[d_i])
        lst[d_i] = tensor
    # end of loop lst[d_i] = tensor([D_i, ... D_0])
    tensor_lst = torch.stack(lst, dim=0)
    return tensor_lst

In [ ]:
df_train = pd.read_csv(project_path+'Datasets/Clarification_Merge_All_Filler_Text_Train_1.csv')
df_val = pd.read_csv(project_path+'Datasets/Clarification_All_Single_Filler_Text_Valid_1.csv')
df_test = pd.read_csv(project_path+'Datasets/Clarification_Merge_All_Filler_Text_Test_1.csv')

In [ ]:
train_texts = df_train['Text_with_options']
train_scores = df_train['ScoreRankList']
val_texts = df_val['Text_with_options']
val_scores = df_val['ScoreRankList']
test_texts = df_test['Text_with_options']
test_scores = df_test['ScoreRankList']

In [ ]:
df_train.columns

Index(['Id', 'Resolved pattern', 'Article title', 'Section header',
       'Previous context', 'Sentence', 'Follow-up context', 'Filler1',
       'Filler2', 'Filler3', 'Filler4', 'Filler5', 'label_1', 'label_2',
       'label_3', 'label_4', 'label_5', 'score_1', 'score_2', 'score_3',
       'score_4', 'score_5', 'Filled_Sentence', 'Filler', 'Label', 'Score',
       'Text', 'Text_with_Filler_Token', 'LabelNum', 'scoreRank_1',
       'scoreRank_2', 'scoreRank_3', 'scoreRank_4', 'scoreRank_5',
       'ScoreRank'],
      dtype='object')

In [ ]:
df_train['Text'][0]#, df_train['Score'][0]

" How to Talk to Your Partner About Their Weigh [SEP] Things You'll Need [SEP] Lose Weight the Healthy Way Use Exercise to Help You Lose Weight [SEP] Lose Weight Fast with stress [SEP] "

In [ ]:
'''
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import re, SnowballStemmer

def clean_text(text):


    text = re.sub(r"\[SEP\]","",text)
    ## Remove puncuation
    text = text.translate(string.punctuation)### IS it code of REMOVING??????

    # replace urls
    re_url = re.compile(r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\
                        .([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*",
                        re.MULTILINE | re.UNICODE)
    # replace ips
    re_ip = re.compile("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}")

    # replace URLs
    #text = re_url.sub("URL", text)

    # replace IPs
    #text = re_ip.sub("IPADDRESS", text)


    ## Convert words to lower case and split them
    text = text.lower().split()

    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 2]

    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    #text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    #text = text.split()
    #stemmer = SnowballStemmer('english')
    #stemmed_words = [stemmer.stem(word) for word in text]
    #text = " ".join(stemmed_words)

    return text

'''

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
'''
train_texts = train_texts.map(lambda x: clean_text(x))
val_texts = val_texts.map(lambda x: clean_text(x))
test_texts = test_texts.map(lambda x: clean_text(x))
'''

In [ ]:
print(train_texts[0])

How to Talk to Your Partner About Their Weigh Lose Weight the Healthy Way Use Exercise to Help You Lose Weight [OPTION_1] Lose Weight Fast with stress [OPTION_2] Lose Weight Fast with plenty [OPTION_3] Lose Weight Fast with strength [OPTION_4] Lose Weight Fast with ease [OPTION_5] Lose Weight Fast with exercise [FOLLOW] 


In [ ]:
import pickle
with open(project_path+'Pickles/Plausibility_Ranking_Generation_LSTM_Datasets/pickle_train_ALL_FILLER_ranking_generation_cleanxy.pickle', 'wb') as f:
  pickle.dump((train_texts,train_scores),f)

In [ ]:
import pickle
with open(project_path+'Pickles/Plausibility_Ranking_Generation_LSTM_Datasets/pickle_valid_ALL_FILLER_ranking_generation_cleanxy.pickle', 'wb') as f:
  pickle.dump((val_texts,val_scores),f)

In [ ]:
import pickle
with open(project_path+'Pickles/Plausibility_Ranking_Generation_LSTM_Datasets/pickle_test_ALL_FILLER_ranking_generation_cleanxy.pickle', 'wb') as f:
  pickle.dump((test_texts,test_scores),f)

In [ ]:
print(train_texts[0])

How to Talk to Your Partner About Their Weigh Lose Weight the Healthy Way Use Exercise to Help You Lose Weight [OPTION_1] Lose Weight Fast with stress [OPTION_2] Lose Weight Fast with plenty [OPTION_3] Lose Weight Fast with strength [OPTION_4] Lose Weight Fast with ease [OPTION_5] Lose Weight Fast with exercise [FOLLOW] 


In [ ]:
#tokenization
import spacy
tok = spacy.load('en')
def tokenize (text):
    return [token.text for token in tok.tokenizer(text)]

In [ ]:
from collections import Counter
#count number of occurences of each word
train_counts = Counter()
for text in train_texts:
    train_counts.update(tokenize(text))

val_counts = Counter()
for text in val_texts:
    val_counts.update(tokenize(text))

test_counts = Counter()
for text in test_texts:
    test_counts.update(tokenize(text))

In [ ]:
print(counts) 

In [ ]:
#creating vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in train_counts:
    vocab2index[word] = len(words)
    words.append(word)

In [ ]:
def encode_sentence(text, vocab2index, Truncate=350):
    tokenized = tokenize(text)
    encoded = np.zeros(Truncate, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(Truncate, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [ ]:
encoded_train_texts = []
for text in train_texts:
  encoded_train_texts.append(np.array(encode_sentence(text,vocab2index)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
import pickle
with open(project_path+'Pickles/Plausibility_Ranking_Generation_LSTM_Datasets/pickle_train_ALL_FILLER_ranking_generation_encoded_1.pickle', 'wb') as f:
  pickle.dump((encoded_train_texts,train_scores),f)

In [ ]:
print(encoded_train_texts[0])

In [ ]:
encoded_val_texts = []
for text in val_texts:
  encoded_val_texts.append(np.array(encode_sentence(text,vocab2index)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
import pickle
with open(project_path+'Pickles/Plausibility_Ranking_Generation_LSTM_Datasets/pickle_val_ALL_FILLER_ranking_generation_encoded_1.pickle', 'wb') as f:
  pickle.dump((encoded_val_texts,val_scores),f)

In [ ]:
encoded_test_texts = []
for text in test_texts:
  encoded_test_texts.append(np.array(encode_sentence(text,vocab2index)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
import pickle
with open(project_path+'Pickles/Plausibility_Ranking_Generation_LSTM_Datasets/pickle_test_ALL_FILLER_ranking_generation_encoded_1.pickle', 'wb') as f:
  pickle.dump((encoded_test_texts,test_scores),f)

In [9]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [ ]:
train_dataset = CustomDataset(encoded_train_texts, train_scores)
val_dataset = CustomDataset(encoded_val_texts, val_scores)
test_dataset = CustomDataset(encoded_test_texts, test_scores)


In [ ]:
import pickle

with open(project_path+'Pickles/Plausibility_Ranking_Generation_LSTM_Datasets/pickle_train_ALL_FILLER_ranking_generation_dataset_1.pickle', 'wb') as f:
  pickle.dump(train_dataset,f)

with open(project_path+'Pickles/Plausibility_Ranking_Generation_LSTM_Datasets/pickle_val_ALL_FILLER_ranking_generation_dataset_1.pickle', 'wb') as f:
  pickle.dump(val_dataset,f)

with open(project_path+'Pickles/Plausibility_Ranking_Generation_LSTM_Datasets/pickle_test_ALL_FILLER_ranking_generation_dataset_1.pickle', 'wb') as f:
  pickle.dump(test_dataset,f)

In [10]:
import pickle

with open(project_path+'Pickles/Plausibility_Ranking_Generation_LSTM_Datasets/pickle_train_ALL_FILLER_ranking_generation_dataset_1.pickle', 'rb') as f:
  train_dataset = pickle.load(f)

with open(project_path+'Pickles/Plausibility_Ranking_Generation_LSTM_Datasets/pickle_val_ALL_FILLER_ranking_generation_dataset_1.pickle', 'rb') as f:
  val_dataset = pickle.load(f)

with open(project_path+'Pickles/Plausibility_Ranking_Generation_LSTM_Datasets/pickle_test_ALL_FILLER_ranking_generation_dataset_1.pickle', 'rb') as f:
  test_dataset = pickle.load(f)

In [ ]:
print(len(train_dataset[0][0]))

350


In [ ]:
def load_glove_vectors(glove_file="/content/drive/My Drive/Machine Learning/Fake News/Evaluation/Evaluation/glove.6B.300d.txt"):
    """Load the glove word vectors"""
    word_vectors = {}
    with open(glove_file,encoding='utf-8') as f:
        for line in f:
            split = line.split()
            word_vectors[split[0]] = np.array([float(x) for x in split[1:]])
    return word_vectors

In [ ]:
def get_emb_matrix(pretrained, word_counts, emb_size = 300):
    """ Creates embedding matrix from word vectors"""
    vocab_size = len(word_counts) + 2
    vocab_to_idx = {}
    vocab = ["", "UNK"]
    W = np.zeros((vocab_size, emb_size), dtype="float32")
    W[0] = np.zeros(emb_size, dtype='float32') # adding a vector for padding
    W[1] = np.random.uniform(-0.25, 0.25, emb_size) # adding a vector for unknown words 
    vocab_to_idx["UNK"] = 1
    i = 2
    for word in word_counts:
        if word in word_vecs:
            W[i] = word_vecs[word]
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
        vocab_to_idx[word] = i
        vocab.append(word)
        i += 1   
    return W, np.array(vocab), vocab_to_idx

In [ ]:
word_vecs = load_glove_vectors()
pretrained_weights, vocab, vocab2index = get_emb_matrix(word_vecs, train_counts)

In [ ]:
with open(project_path+'Pickles/Plausibility_Ranking_Generation_LSTM_Datasets/pickle_glove_weights_ALL_FILLER_ranking_generation_dataset_1.pickle', 'wb') as f:
  pickle.dump(pretrained_weights,f)

In [11]:
with open(project_path+'Pickles/Plausibility_Ranking_Generation_LSTM_Datasets/pickle_glove_weights_ALL_FILLER_ranking_generation_dataset_1.pickle', 'rb') as f:
  pretrained_weights = pickle.load(f)

In [ ]:
print(pretrained_weights.shape,len(vocab))

(19783, 300) 19783


In [12]:
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class CNN_glove_vecs(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights,seq_len) :
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
        self.embeddings.weight.requires_grad = False ## freeze embeddings


        self.kernel_1 = 2
        self.stride = 2

        #self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True,bidirectional = True,)
        self.conv_1 = nn.Conv1d(seq_len, self.hidden_dim, self.kernel_1, self.stride)
        self.pool_1 = nn.MaxPool1d(self.kernel_1,self.stride)


        self.linear1 = nn.Linear(75, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, 5)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.embeddings(x)

        #packed_x = pack_padded_sequence(x,l,batch_first=True,enforce_sorted=False)
        #print(packed_x)

        cnn_out = self.conv_1(x)
        cnn_out = self.pool_1(cnn_out)[:,0,:]
        #print(len(packed_lstm_out[0]))
        #lstm_out,_ = pad_packed_sequence(packed_lstm_out,batch_first=True)
        #print(lstm_out.shape)
        #print(_)
        #out = torch.cat((lstm_out[:,0,:self.hidden_dim],lstm_out[:,0,self.hidden_dim:]),dim=1)
        #print(out.shape)
        out = self.linear1(cnn_out)
        out = self.relu(out)
        out = self.linear2(out)
        #print(out)

        return out

In [14]:
seq_len = 350

models = [0,0,0,0,0]
models[0] = CNN_glove_vecs(pretrained_weights.shape[0], pretrained_weights.shape[1], 50, pretrained_weights,seq_len)
models[1] = CNN_glove_vecs(pretrained_weights.shape[0], pretrained_weights.shape[1], 50, pretrained_weights,seq_len)
models[2] = CNN_glove_vecs(pretrained_weights.shape[0], pretrained_weights.shape[1], 50, pretrained_weights,seq_len)
models[3] = CNN_glove_vecs(pretrained_weights.shape[0], pretrained_weights.shape[1], 50, pretrained_weights,seq_len)
models[4] = CNN_glove_vecs(pretrained_weights.shape[0], pretrained_weights.shape[1], 50, pretrained_weights,seq_len)

In [ ]:
print(train_dataset[0].X)

In [17]:
import ast

In [ ]:
for idx,batch in enumerate(train_dl):
    x = batch[0].long()
    y = batch[1]
    #y_pred = model(x, batch[2]).squeeze(-1)

    #y_p = torch.max(y_pred, 1)[1]
    #print(y_pred)
    print(len(y))
    y_ = ast.literal_eval(y[0])
    print(y_[0])
    break

512
1


In [15]:
from torch.utils.data import DataLoader

batch_size = 512
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=batch_size)
test_dl = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
import torch.nn.functional as F

loss = F.mse_loss(torch.FloatTensor([4.34,4.5]), torch.FloatTensor([4.3,4.5]))
print(loss)

tensor(0.0008)


In [ ]:
'''

def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1] #########################
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

'''

In [ ]:
dic = {'a':1,'b':2}
for k,v in dic.items():
  print(k,v)

a 1
b 2


In [16]:
from sklearn.metrics import accuracy_score,f1_score
from scipy.stats import spearmanr

def compute_metric(y_true,y_pred):
  result_dic = {'acc':0,'spearmanr':0}
  #print(len(y_true))
  #print(len(y_pred))

  result_dic['acc'] = ((y_true==y_pred).sum())/len(y_true)
  #result_dic['f1_micro'] = f1_score(y_true, y_pred, average='micro')
  #result_dic['f1_weighted'] = f1_score(y_true, y_pred, average='weighted')
  result_dic['spearmanr'] = spearmanr(y_true,y_pred)[0]

  return result_dic



In [18]:
import ast

import torch.nn.functional as F

def train_model_regr(model, epochs=10, lr=0.001, log_every_epoch = 1,device=None, val_dataloader=None, save_model = False,PATH = None):
    for model in models:
      model.to(device)
    parameters = filter(lambda p: p.requires_grad, models[0].parameters())
    parameters = filter(lambda p: p.requires_grad, models[1].parameters())
    parameters = filter(lambda p: p.requires_grad, models[2].parameters())
    parameters = filter(lambda p: p.requires_grad, models[3].parameters())
    parameters = filter(lambda p: p.requires_grad, models[4].parameters())
    optimizer = torch.optim.Adam(list(models[0].parameters()) + list(models[1].parameters()) +list(models[2].parameters()) +list(models[3].parameters()) +
                                 list(models[4].parameters()), lr=lr)
    for i in range(epochs):
        for model in models:
          model.train()
        sum_loss = 0.0
        total = 0
        for x, y_init, _ in train_dl:
            optimizer.zero_grad()
            x = x.long().to(device)
            #y = y.to(device)
            #y_pred = [0, 0, 0, 0, 0]
            loss = 0.0
            y = []
            for ind in range(len(y_init)):
              y.append(torch.LongTensor(ast.literal_eval(y_init[ind])))
              #l = l.to(device)
            y = tensorify(y)
            y = y.long()
            y = y.to(device)

            #print(y[:,0].unsqueeze(-1).size())
            #y_pred = models[0](x, l)
            #print(y_pred.size())

            for k in range(5):
              y_pred = models[k](x)
              #y = torch.LongTensor(y)
              loss += F.cross_entropy(y_pred, y[:,k]) #################

            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, dic = validation_metrics_regr(model, val_dl,is_compute_metric=True,device=device)
        if i % log_every_epoch == 0:
            print('Epoch: '+str(i))
            print("train loss %.3f, val loss %.3f" % (sum_loss/total, val_loss))
            if(dic is not None):
              for k,v in dic.items():
                print(str(k)+': '+str(v),end='\t')
              print('')
            
            if(save_model):
              for modelNo,model in enumerate(models):
                torch.save(model.state_dict(), PATH+'CNN_Rank_Gen_ALL_FILLER_state_dict_model_No_'+str(modelNo)+'_ep_'+str(i)+'_1.pt')

def validation_metrics_regr (model, valid_dl,is_compute_metric=True,device=None):
    #model = model.to(device)
    for model in models:
      model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    total_dic = {}
    with torch.no_grad():
      for x, y_init, _ in valid_dl:
        x = x.long().to(device)
        #y = y.float().to(device)
        #y = y.to(device)
        #y_pred = [0, 0, 0, 0, 0]
        loss = 0.0
        y = []
        for ind in range(len(y_init)):
          y.append(torch.LongTensor(ast.literal_eval(y_init[ind])))
          #l = l.to(device)
        y = tensorify(y)
        y = y.long()
        y = y.to(device)

        #print(y[:,0].unsqueeze(-1).size())
        #y_pred = models[0](x, l)
        #print(y_pred.size())

        y_hat = []
        for k in range(5):
          y_pred = models[k](x,)
          #y = torch.LongTensor(y)
          loss += F.cross_entropy(y_pred, y[:,k]) #################
          y_hat.append(torch.argmax(y_pred,dim=1))

        sum_loss += loss.item()*y.shape[0]

        y_hat = tensorify(y_hat)
        y_hat = y_hat.long()
        y_hat = y_hat.T
        y_hat = y_hat.to(device)
        #print(y_hat.size(), y.size())
        y_hat = y_hat.ravel()
        y = y.ravel()
        #print(y_hat.size(), y.size())
        y = y.detach().cpu().numpy()
        y_hat = y_hat.detach().cpu().numpy()

        total += y.shape[0]
        
        if(is_compute_metric==True):
          dic = compute_metric(y,y_hat)
          for k,v in dic.items():
            if k not in total_dic.keys():
              total_dic[k]=0
            total_dic[k]+=v*y.shape[0]
        for k,v in total_dic.items():
          total_dic[k] = v/total
    return sum_loss/total,total_dic

In [21]:
seq_len = 350

models = [0,0,0,0,0]
models[0] = CNN_glove_vecs(pretrained_weights.shape[0], pretrained_weights.shape[1], 50, pretrained_weights,seq_len)
models[1] = CNN_glove_vecs(pretrained_weights.shape[0], pretrained_weights.shape[1], 50, pretrained_weights,seq_len)
models[2] = CNN_glove_vecs(pretrained_weights.shape[0], pretrained_weights.shape[1], 50, pretrained_weights,seq_len)
models[3] = CNN_glove_vecs(pretrained_weights.shape[0], pretrained_weights.shape[1], 50, pretrained_weights,seq_len)
models[4] = CNN_glove_vecs(pretrained_weights.shape[0], pretrained_weights.shape[1], 50, pretrained_weights,seq_len)

In [22]:
path = project_path +'Saved_Models/PLAUSIBILITY_CNN_MULTIOUTPUT_1/'
train_model_regr(models, epochs=100, lr=0.05,device=device,val_dataloader=test_dl, save_model=True, PATH=path)

Epoch: 0
train loss 8.980, val loss 1.626
acc: 0.20551378446115287	spearmanr: -0.02431934316721479	
Epoch: 1
train loss 8.116, val loss 1.613
acc: 0.19598997493734335	spearmanr: -0.023476862259996963	
Epoch: 2
train loss 8.045, val loss 1.604
acc: 0.21804511278195488	spearmanr: -0.037688244525042505	
Epoch: 3
train loss 8.009, val loss 1.602
acc: 0.23709273182957394	spearmanr: 0.02654502375062947	
Epoch: 4
train loss 8.001, val loss 1.603
acc: 0.23258145363408522	spearmanr: 0.033797582762530025	


/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Epoch: 5
train loss 8.003, val loss 1.604
acc: 0.23258145363408522	spearmanr: nan	
Epoch: 6
train loss 8.008, val loss 1.604
acc: 0.23258145363408522	spearmanr: nan	
Epoch: 7
train loss 8.012, val loss 1.603
acc: 0.23258145363408522	spearmanr: nan	
Epoch: 8
train loss 8.008, val loss 1.599
acc: 0.23258145363408522	spearmanr: nan	
Epoch: 9
train loss 8.004, val loss 1.599
acc: 0.23258145363408522	spearmanr: nan	
Epoch: 10
train loss 8.007, val loss 1.603
acc: 0.23258145363408522	spearmanr: nan	
Epoch: 11
train loss 8.010, val loss 1.607
acc: 0.23258145363408522	spearmanr: nan	
Epoch: 12
train loss 8.018, val loss 1.607
acc: 0.23258145363408522	spearmanr: nan	
Epoch: 13
train loss 8.011, val loss 1.602
acc: 0.23258145363408522	spearmanr: nan	
Epoch: 14
train loss 8.005, val loss 1.600
acc: 0.23258145363408522	spearmanr: nan	
Epoch: 15
train loss 8.008, val loss 1.602
acc: 0.22055137844611528	spearmanr: 0.03123595670863303	
Epoch: 16
train loss 8.004, val loss 1.601
acc: 0.232581453634085

In [24]:
seq_len = 350

models = [0,0,0,0,0]

for k in range(5):
  models[k] = CNN_glove_vecs(pretrained_weights.shape[0], pretrained_weights.shape[1], 50, pretrained_weights,seq_len)

  model_state_epoch = 'CNN_Rank_Gen_ALL_FILLER_state_dict_model_No_'+str(k)+'_ep_'+str(79)+'_1.pt'
  path = project_path +'Saved_Models/PLAUSIBILITY_CNN_MULTIOUTPUT_1/'+ model_state_epoch
  models[k].load_state_dict(torch.load(path))
  models[k].to(device)

validation_metrics_regr(models, val_dl,is_compute_metric=True,device=device)

(1.599940299987793,
 {'acc': 0.23609022556390977, 'spearmanr': 0.04947296944752308})

In [ ]:
!pip install networkx

In [ ]:
import networkx as nx

G = nx.Graph([(1,2),(1,3),(1,4),(2,3),(2,4),(3,4),(1,5)])
nx.k_core(G,k=2).nodes()

NodeView((1, 2, 3, 4))